In [1]:
import numpy as np 
from tqdm import tqdm as notebook_tqdm

import torch
import torchvision
from torch import nn, optim
from torch.nn import init
from torchvision import datasets, transforms


/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [3]:
def get_loaders(batch_size):
    transformer = transforms.Compose([torchvision.transforms.ToTensor()])
    train_loader = torch.utils.data.DataLoader(datasets.MNIST('./files/', train=True, download=True, transform=transformer),
                                               batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(datasets.MNIST('./files/', train=False, download=True, transform=transformer),
                                                  batch_size=batch_size, shuffle=True)
    
    return train_loader, test_loader

def save_model(mstate, optimizer, epoch, name):
    torch.save({'model_state_dict': mstate,
                'optimizer': optimizer.state_dict(),
                'epoch': epoch},
                f'model_{name}.pt')

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5, stride=2),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(10, 20, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Dropout2d(),
            nn.Flatten()
        )
        self.fc1 = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout()
        )
        self.fc2 = nn.Linear(50, 10)
        
        self.apply(self._init_weights)
        
    def forward(self, x, f):
        if f: print(f'\nIn Model: input size {x.size()}')
            
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x
    
    def _init_weights(self, m):
        if isinstance(m, (torch.nn.Conv2d, torch.nn.Linear)):
            init.kaiming_normal_(m.weight.data, a=0, mode='fan_in', nonlinearity='relu')
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)

In [5]:
def train_DDP(batch_size: int = 64, n_epochs=1):
    device = torch.device( f'cuda:{0}' if torch.cuda.is_available() else 'cpu')
    model = Net()
    optimizer = optim.Adam(model.parameters(), lr=1e-2)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    if torch.cuda.device_count() > 1:
        print(f'There are: {torch.cuda.device_count()} GPUs!')
        model = torch.nn.DataParallel(model, device_ids=[0,1])
        model.to(device)        
    else:
        model.to(device)
    
    train_loader, test_loader = get_loaders(batch_size)
    highest_acc = 0
    for epoch in range(1, n_epochs + 1):
        train_bar = notebook_tqdm(train_loader)
        model.train()
        train_losses = []
        for batch_idx, (data, target) in enumerate(train_bar):
            optimizer.zero_grad()
                        
            f = True if not train_losses and epoch ==1 else False    
            output = model(data.to(device), f)
            loss = loss_fn(output, target.to(device))
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            train_bar.set_description(f'Train Epoch: {epoch} Loss: {np.mean(train_losses):.6f}')
        

        model.eval()
        acc = []
        test_bar = notebook_tqdm(test_loader)
        for data, target in test_bar:
            with torch.no_grad():
                output = model(data.to(device), False)
            acc.extend((torch.argmax(output.softmax(1), dim=1) == target.to(device)).data.cpu().numpy())
            test_bar.set_description(f'Test set: Accuracy: {100. * np.mean(acc):.0f}%')
        
        
        if np.mean(acc) > highest_acc:
            highest_acc = np.mean(acc)

            if torch.cuda.device_count() > 1:
                mstate = model.module.state_dict()
            else:
                mstate = model.state_dict()
            save_model(mstate, optimizer, epoch, 'DDP')
            
            
train_DDP(batch_size =250, n_epochs=5)

There are: 2 GPUs!


  0%|                                                   | 0/240 [00:00<?, ?it/s]


In Model: input size torch.Size([125, 1, 28, 28])

In Model: input size torch.Size([125, 1, 28, 28])


Test set: Accuracy: 97%: 100%|██████████████████| 40/40 [00:01<00:00, 32.53it/s]
